# Dog Breed Identification by Image Classification


The goal of this project is to be able to identify the breed of a dog given a picture of it. The data was provided by Kaggle for their dog breed identification contest. My approach is to use a convolutional neural network to classify these images. I will train a variety of networks of my own design and additionally apply some pre-trained networks and compare the results. 

In [1]:
#import relevant packages
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Input, Activation
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam
from keras.applications import VGG19
from keras.applications import ResNet50
from keras import Model
import cv2
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/home/connor/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#load the data
labels = pd.read_csv('data/labels.csv.zip',compression='zip')
labels.head(2)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo


In [3]:
#The images are all different dimensions so this will be the parameter to
#resize them to be square 
image_size = 128

### Preparing the data to train the networks

In [4]:
def prepare_train_validation():
    #converting breed categories to categorical booleans
    one_hot_labels = pd.get_dummies(labels['breed']).values
    X_raw = []
    y_raw = []
    #loads and processes the images into image_size by image_size by 3 tensors
    for ind, row in enumerate(labels.values):
        img = cv2.imread('data/train/{}.jpg'.format(row[0]))
        img = cv2.resize(img,(image_size,image_size))
        X_raw.append(img)
        y_raw.append(one_hot_labels[ind,:])
    X = np.array(X_raw)
    y =np.array(y_raw)
    X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2,stratify = y, random_state=1)
    return X_train, X_validation, y_train, y_validation

In [5]:
#creates training and validation sets with every class represented
#in both the training and validation sets.
X_train, X_validation, y_train, y_validation = prepare_train_validation()


In [6]:
#length of training data for use in determining batch size later
m_training = X_train.shape[0]

## First Experimental Network
this network is based on VGG16 but has been simplified 

In [7]:
def conv_network_1( input_shape,weights_path = None):
    model = Sequential()
    
    model.add(ZeroPadding2D((1,1),input_shape=input_shape))
    
    model.add(Convolution2D(32,(3,3),activation ='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(32,(3,3),activation ='relu'))
    #model.add(MaxPooling2D((2,2),strides = (2,2)))
    
    model.add(Convolution2D(64,(3,3),activation ='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64,(3,3),activation ='relu'))
    #model.add(MaxPooling2D((2,2),strides = (2,2)))
    
    model.add(Convolution2D(128,(3,3),activation ='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128,(3,3),activation ='relu'))
    model.add(MaxPooling2D((2,2),strides = (2,2)))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu',name="dense1"))
    model.add(Dense(1024, activation='relu',name="dense2"))
    model.add(Dense(120, input_shape = (10222, 10342), activation= 'softmax',name="dense3"))
    return model

In [8]:
#Training the network using an rmsprop optimizer
model = conv_network_1(input_shape = (image_size,image_size,3))
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',metrics = ['accuracy'] )
model.fit(X_train,y_train,epochs = 10, batch_size = 38)

Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[492032,1024]
	 [[Node: training/RMSprop/gradients/dense1/MatMul_grad/MatMul_1 = MatMul[T=DT_FLOAT, _class=["loc:@dense1/MatMul"], transpose_a=true, transpose_b=false, _device="/job:localhost/replica:0/task:0/cpu:0"](flatten_1/Reshape, training/RMSprop/gradients/dense1/Relu_grad/ReluGrad)]]

Caused by op 'training/RMSprop/gradients/dense1/MatMul_grad/MatMul_1', defined at:
  File "/home/connor/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/connor/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/connor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/connor/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/connor/anaconda3/lib/python3.6/site-packages/ipykernel/k

In [ ]:
for layer in model.layers:
    print(layer.get_output_at(0).get_shape().as_list())
#model.fit(X,y,epochs = 10, batch_size = 38)

In [ ]:
def conv_network_2(input_shape):
    X_input = Input(input_shape)
    X = ZeroPadding2D((1,1),input_shape=input_shape)(X_input)
    
    X = Convolution2D(32,(5,5),activation ='relu',name='conv1a')(X)
    X = ZeroPadding2D((1,1))(X)
    X = Convolution2D(32,(5,5),activation ='relu',name='conv1b')(X)
    X = MaxPooling2D((2,2),strides = (2,2),name = 'pool1a')(X)
    
    X = Convolution2D(64,(3,3),activation ='relu',name='conv2a')(X)
    X = ZeroPadding2D((1,1))(X)
    X = Convolution2D(64,(3,3),activation ='relu',name='conv2b')(X)
    X = MaxPooling2D((2,2),strides = (2,2),name = 'pool2a')(X)
    
    X = Convolution2D(128,(3,3),activation ='relu',name='conv3a')(X)
    X = ZeroPadding2D((1,1))(X)
    X = Convolution2D(128,(3,3),activation ='relu',name='conv3b')(X)
    X = MaxPooling2D((2,2),strides = (2,2),name = 'pool3a')(X)
    X = Activation('relu')(X)
    
    X = Flatten()(X)
    X = Dense(120, activation='relu',name="dense1")(X)

    model = Model(inputs = X_input, outputs = X, name = 'convnet2')
    return model

In [ ]:
#model2 = conv_network_2(X.shape[1:])

In [ ]:
#model2.compile(optimizer = 'rmsprop',loss = 'categorical_crossentropy',)

In [ ]:
#model2.fit(X,y,epochs = 10, batch_size = 38)